In [ ]:
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']='1'
import numpy as np
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Embedding, Dropout, PReLU
from keras.layers import Bidirectional, SpatialDropout1D, CuDNNGRU
from keras.regularizers import l2
from toxic.keras_utils import Capsule
from toxic.NN import NN

In [ ]:
class CapsuleNet(NN):
    """Input-> Embedding-> GRU-> Capsule-> Output"""

    def __init__(self, model_name, embedding_file, max_seq_len=250):

        super().__init__(model_name, max_seq_len, file_dir='../../input',
                         word_embedding_file=embedding_file)
        self.load_labels()
        self.load_word_sequences()
        self.load_word_vectors()

    def get_model(self, n_capsule = 10, n_routings = 5, capsule_dim = 16,
         n_recurrent=100, dropout_rate=0.2, l2_penalty=0.0001):
        K.clear_session()

        inputs = Input(shape=(self.max_seq_len,))
        x = Embedding(self.word_embedding_matrix.shape[0], 300, 
                      weights=[self.word_embedding_matrix], trainable=False)(inputs)        
        x = SpatialDropout1D(dropout_rate)(x)
        x = Bidirectional(
            CuDNNGRU(n_recurrent, return_sequences=True,
                     kernel_regularizer=l2(l2_penalty),
                     recurrent_regularizer=l2(l2_penalty)))(x)
        x = PReLU()(x)
        x = Capsule(
            num_capsule=n_capsule, dim_capsule=capsule_dim,
            routings=n_routings, share_weights=True)(x)
        x = Flatten()(x)
        x = Dropout(dropout_rate)(x)
        outputs = Dense(6, activation='sigmoid')(x)
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(loss='binary_crossentropy', optimizer='nadam')
        return model

In [ ]:
param_lims = {
    'n_capsule':    (6, 14),   
    'capsule_dim':  (16, 20),
    'n_routings':   (3, 7), 
    'n_recurrent':  (100, 160),
    'dropout_rate': (0, 0.25),  
    'l2_penalty':   (-5, -2), 
}   

for emb in ['toxic', 'fasttext', 'glove', 'lexvec']:
    nn = CapsuleNet(
        model_name = f'CapsuleNet_{emb}',
        embedding_file = f'embeddings/{emb}_embeddings.txt')
    for i in np.random.randint(10000, size=10):
        nn.predict_kfold(param_lims, n_folds=5, seed=0, monitor_training=False)